# EE932: Programmming Assignment 3
**Name**:                               </br>
**Roll No.**:


# Cliff Walking 

Through this gridworld exercise we will compare Sarsa and Q-learning algorithms, highlighting the difference between them. Consider the Cliff World shown in the assignment. This is a standard undiscounted, episodic task, with start and goal states, and the usual actions causing movement up, down, right, and left. Reward is −1 on all transitions except those into the the region marked “The Cliff.” Stepping into this region incurs a reward of −100 and sends the agent instantly back to the start. There are totally 48 states numbered from 0 to 47



In [ ]:
#!pip install pygame
#!pip install "gymnasium[toy-text]"

In [ ]:
# Import
%matplotlib inline
import gymnasium as gym
import itertools
import matplotlib
import numpy as np
import pandas as pd
import sys
import time
from matplotlib import pyplot as plt
from collections import defaultdict
matplotlib.style.use('ggplot')

In [ ]:
# Create environment
# change render mode to 'ansi' to run the algorithm fast
env = gym.make('CliffWalking-v0', render_mode='ansi')
env.reset()
env.render()  
time.sleep(0.1)

In [ ]:
env.step(1)
# state, action, terminated?, truncated?, info

In [ ]:
# Allowed Actions
# 0 => UP
# 1 => Right
# 2 => Down
# 3 => Left

# Let us take a few actions and get familiar with the environment
print('We started here')
env.reset()
env.render()

# Let us take up first
print('Let us take UP action')
env.step(0)
env.render()

# Now lets take another UP
print('Another UP action')
env.step(0)
env.render()

# Now lets take two rights
print('Two Right actions')
env.step(1)
env.step(1)
env.render()

# Now one left
print('One Left action')
env.step(3)
env.render()

env.close()

In [ ]:
# A helper function for plotting the comparision graphs between two or more algorithms. 
# You need not understand this code.
# Just look at the next cell to understand how to use it.

def plot_episode_reward(a, stats, smoothing_window=10, noshow=False):
    # Plot the episode reward over time
    fig2 = plt.figure(figsize=(7,3))
    for stat in stats:
        rewards_smoothed = pd.Series(stat).rolling(smoothing_window, min_periods=smoothing_window).mean()
        plt.plot(rewards_smoothed)
    plt.xlabel("Episode")
    plt.ylabel("Episode Reward (Smoothed)")
    plt.title("Episode Reward over Time (Smoothed over window size {})".format(smoothing_window))
    plt.legend(a)
    if noshow:
        plt.close(fig2)
    else:
        plt.show(fig2)

In [ ]:
# Demo for using Plotting helper function
# Here we are comparing two algorithms which were run for 3 episodes each. 

stats_algo1= [10,12,4]    #The return obtained in each episode for Algo1
stats_algo2= [-1,12,90]    #The return obtained in each episode for Algo2

#Plotting the comparision
#Here the last argument is to smoothen the plot. Use higher values such as 10 for smooth plots.

plot_episode_reward(['Algo1','Algo2'],[stats_algo1,stats_algo2],smoothing_window=1)  

### Epsilon-greedy policy from Q-function and epsilon

Helper function to create epsilon-greedy policy based on a given Q-function and epsilon.

In [ ]:
def make_epsilon_greedy_policy(Q, epsilon, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function and epsilon.
    
    Args:
        Q: A dictionary that maps from state -> action-values.
            Each value is a numpy array of length nA (see below)
        epsilon: The probability to select a random action. Float between 0 and 1.
        nA: Number of actions in the environment.
    
    Returns:
        A function that takes the state/observation as an argument and returns
        the probabilities for each action in the form of a numpy array of length nA.
    
    """
    def policy_fn(observation):
        A = np.ones(nA, dtype=float) * epsilon / nA
        
        if type(observation) is tuple:
            best_action = np.argmax(Q[observation[0]])
        else:
            best_action = np.argmax(Q[observation])
            
        A[best_action] += (1.0 - epsilon)
        return A
    return policy_fn

In [ ]:
# How to use make_epsilon-greedy_policy function
# consider two states s1, s2 and 3 actions {0,1,2} in each state

Q={'s1':[0.5, 2, 3], 's2':[-0.1, 4 , 3]}  # A sample Q-function Q(s,a) for each state-action pair

# For this example it is clear that for state s1, action 2 is the greedy action.
# For state s2, action 1 is the greedy action

"""
If we want an epsilon greedy policy with epsilon=0.3, then the best action
should get a probility = 0.7 + 0.3/3 = 0.8
other action with probability = 0.3/3 = 0.1

"""

# Let us verify this using our make_epsilon-greedy_policy function

policy_fn = make_epsilon_greedy_policy(Q, epsilon=0.3, nA=3)

print("The epsilon-greedy policy at state s1 is ", policy_fn('s1'))
print("The epsilon-greedy policy at state s2 is ", policy_fn('s2'))

***

## Q1.1 SARSA algorithm

Implement the SARSA algorithm using the following function template that retunts a tuple containing the optimal action value function and training statistics (array of cumulatetive rewards at each episode) **[3.5 Marks]**

In [ ]:
def sarsa(env, num_episodes, discount_factor=1.0, alpha=0.5, epsilon=0.1):
    """
    SARSA algorithm: Finds the optimal epsilon-greedy policy.
    
    Args:
        env: OpenAI environment.
        num_episodes: Number of episodes to run for.
        discount_factor: Gamma discount factor.
        alpha: TD learning rate.
        epsilon: Chance the sample a random action. Float betwen 0 and 1.
    
    Returns:
        A tuple (Q, stats).
        Q is the optimal action-value function, a dictionary mapping state -> action values.
        stats is an EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
    """
    
    # The final action-value function.
    # A nested dictionary that maps state -> (action -> action-value).
    Q = defaultdict(lambda: np.ones(env.action_space.n))
    
    # Keeps track of cumulative reward at each episode 
    stats = []
    
    for i_episode in range(num_episodes):
        # Print out which episode we're on, useful for debugging.
        if (i_episode + 1) % 10 == 0:
            print("\rEpisode {}/{}.".format(i_episode + 1, num_episodes), end="")
            sys.stdout.flush()
        
        # The policy we're following
        policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
    
        # Reset the environment and pick the first action
        state = env.reset()[0]
        action_probs = policy(state)
        action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
            
        total_reward=0
        
        # One step in the environment
        for t in itertools.count():
            # Generate the policy we're following from action value function
            policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
            
            # Select an action at the current state and take a step
            next_state, reward, done, _, _ = env.step(action)
            total_reward += reward
            
            # Pick the next action
            next_action_probs = policy(next_state)
            next_action = np.random.choice(np.arange(len(next_action_probs)), p=next_action_probs)
            
            # TD Update: SARSA 
            total_reward = 0       
            
            # break the loop if episode terminates
            if done:
                break
            
            # update the action and state variables
            action = next_action
            state = next_state
        
        # store the cumulative episode reward
        stats.append(total_reward)
    
    return Q, stats

In [ ]:
episodes = 200
Q_sarsa, stats_sarsa = sarsa(env, episodes, epsilon=0.1)
plot_episode_reward(['SARSA'],[stats_sarsa])

## Q1.2 Q-Learning algorithm

Implement the Q-Learning algorithm using the following function template that retunts a tuple containing the optimal action value function and training statistics (array of cumulatetive rewards at each episode) **[3.5 Marks]**

In [ ]:
def q_learning(env, num_episodes, discount_factor=1.0, alpha=0.5, epsilon=0.1):
    """
    Q-Learning algorithm: Finds the optimal greedy policy
    while following an epsilon-greedy policy
    
    Args:
        env: OpenAI environment.
        num_episodes: Number of episodes to run for.
        discount_factor: Gamma discount factor.
        alpha: TD learning rate.
        epsilon: Chance to sample a random action. Float between 0 and 1.
    
    Returns:
        A tuple (Q, episode_lengths).
        Q is the optimal action-value function, a dictionary mapping state -> action values.
        stats is an EpisodeStats object with two numpy arrays for episode_lengths and episode_rewards.
        target_rollout_stats is an EpisodeStats object with two numpy arrays for target_rollout_episode_lengths and target_rollout_episode_rewards.
    """
    
    # The final action-value function.
    # Initialize a nested dictionary that maps state -> (action -> action-value).
    Q = defaultdict(lambda: np.zeros(env.action_space.n))

    # Keeps track of cumulative reward at each episode 
    stats = []
        
    for i_episode in range(num_episodes):
        # Print out which episode we're on, useful for debugging.
        if (i_episode + 1) % 10 == 0:
            print("\rEpisode {}/{}.".format(i_episode + 1, num_episodes), end="")
            sys.stdout.flush()
        
        # The policy we're following
        policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
        
        # Reset the environment
        state = env.reset()[0]        
        
        # One step in the environment
        total_reward = 0
        for t in itertools.count():
            # The policy we're following
            policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
            
            # Take a step
            action_probs = policy(state)
            action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
            next_state, reward, done, _, _ = env.step(action)

            total_reward += reward

            # TD Update: Q-Learning
            # write your code here

            # break the loop if episode terminates
            if done:
                break 
            
            # update the state
            state = next_state
        
        # store the cumulative episode reward
        stats.append(total_reward)
    
    return Q, stats

### Comparison and Generating Plots 
Plotting the reward over episodes for both SARSA and Q-Learning

In [ ]:
episodes = 10
Q_q, stats_q = q_learning(env, episodes, epsilon=0.2)
Q_sarsa, stats_sarsa = sarsa(env, episodes, epsilon=0.2)
plot_episode_reward(['Q-Learning Behavior Policy','SARSA'],[stats_q,stats_sarsa])

## Before we go on to visualize the policies generated by two TD methods answer the following questions:

#### Q2. Why is Q-learning considered an off-policy control method? **[2 Marks]** <br>
#### Q3. Which algorithm takes a more conservative path? **[1 Marks]** <br>
#### Q4. **(Bonus)** Suppose while learning the action selection is greedy. Is Q-learning then exactly the same algorithm as Sarsa? Will they make exactly the same action selections and weight updates? **[2 Marks]**


### Answer:

### Visualizing the policy

In [ ]:
# function to visualize policy over the environment
def render_policy(pi, mode='human', close=False):

    cliff_pose = [(3,1),(3,2),(3,3),(3,4),(3,5),(3,6),(3,7),(3,8),(3,9),(3,10)] # cliff states
    for s in range(48):
        position = np.unravel_index(s, (4,12))
        # print(self.s)
        if pi[position] == 0:
            output = " ↑ "
        elif pi[position] == 1:
            output = " → "
        elif pi[position] == 2:
            output = " ↓ "
        elif pi[position] == 3:
            output = " ← "

        if position == (3,11):
            output = " T "
        elif position in cliff_pose:
            output = " G "

        if position[1] == 0:
            output = output.lstrip()
        if position[1] == 12 - 1:
            output = output.rstrip()
            output += "\n"

        print(output, end="")
    print("\n")

In [ ]:
# extracting greedy policy from the Q function
pi_sarsa = np.zeros(env.observation_space.n)
pi_q = np.zeros(env.observation_space.n)
for s in range(env.observation_space.n):
    pi_sarsa[s] = np.argmax(Q_sarsa[s]) 
    pi_q[s] = np.argmax(Q_q[s])
    
pi_sarsa = np.reshape(pi_sarsa, env.shape)
pi_q = np.reshape(pi_q, env.shape)

print('SARSA policy:')
render_policy(pi_sarsa)
print('Q-learning policy:')
render_policy(pi_q)